# Application Programming Interface (API) Examples

This notebook demonstrates how to use the **Gherkin Processor** via the *application programming interface* (*API*).

> Note: This notebook assumes that the **Gherkin Processor** package is already installed. See the *[Installation](../README.md/#installation)* guide for details.

## Prerequisites

The modules are imported. (For the Notebook, the example scenarios are also imported.)

In [1]:
from gherkin_processor.utils import scenario    # file
from gherkin_processor.scenario import Scenario # class

simple_example_text = open("examples/data/simple_example.feature", "r").read()
complex_example_text = open("examples/data/complex_example.feature", "r").read()
invalid_step_order_example_text = open("examples/data/invalid_step_order_example.feature", "r").read()
invalid_table_example_text = open("examples/data/invalid_table_example.feature", "r").read()

## Validation

The Gherkin scenarios can be validated in different ways. This includes simple `boolean` determination, issue description and optional description.

### Validation via `boolean`

For this, the `scenario.is_valid(str): bool` function can be used in the utilities.

In [2]:
print(f"{type(scenario.is_valid(simple_example_text))}: {scenario.is_valid(simple_example_text)}")                         # <class 'bool'>: True
print(f"{type(scenario.is_valid(invalid_step_order_example_text))}: {scenario.is_valid(invalid_step_order_example_text)}") # <class 'bool'>: False

<class 'bool'>: True
<class 'bool'>: False


### Validation via issue description

For this, the `scenario.issue_description(str): str` function can be used in the utilities. It will always return a string, if the scenario is valid, then the string is empty.

In [3]:
print(f"{type(scenario.issue_description(simple_example_text))}: {scenario.issue_description(simple_example_text)}")                         # <class 'str'>: 
print(f"{type(scenario.issue_description(invalid_step_order_example_text))}: {scenario.issue_description(invalid_step_order_example_text)}") # <class 'str'>: Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".

<class 'str'>: 
<class 'str'>: Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".


### Validation via optional issue description

For this, the `scenario.validate(str): str | None` function can be used in the utilities. It only return a string, if the scenario is invalid, otherwise it returns `None`.

In [4]:
print(f"{type(scenario.validate(simple_example_text))}: {scenario.validate(simple_example_text)}")                         # <class 'NoneType'>: None
print(f"{type(scenario.validate(invalid_step_order_example_text))}: {scenario.validate(invalid_step_order_example_text)}") # <class 'str'>: Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".

<class 'NoneType'>: None
<class 'str'>: Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".


## Processing

Gherkin scenarios can be processed from a single text (`str`) into a `Scenario` dataclass. For the processing, both the utility processor and the dataclass constructor can be used.

### Utility processor

Uses the `scenario.process(str, bool=True)` function. The first parameter is the scenario as a text and the second determines whether to validate the scenario syntax.

#### Validation

When validation is set, the processor will raise an error with the issue description as the message. The processor can also validate specific criteria, such as incorrect parameter type, empty scenario text, etc.

In [5]:
try: scenario.process(None, True)
except TypeError as e: print(e) # Parameter 'scenario_text' is not string type.
try: scenario.process("")
except ValueError as e: print(e) # Scenario is empty.
try: scenario.process("# Only comment")
except ValueError as e: print(e) # Scenario only contains empty lines or comments.
try: scenario.process(invalid_step_order_example_text)
except ValueError as e: print(e) # Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".

Parameter 'scenario_text' is not string type.
Scenario is empty.
Scenario only contains empty lines or comments.
Prohibited keyword in 'GIVEN step table' at line [7]: "Then the <Type> pizza is ready to eat".


#### No validation

When validation is not set, the processor will try to process the scenario as much as possible. It does not catch any internal error raise.

In [6]:
empty_scenario = scenario.process("", False)
comment_only_scenario = scenario.process("# Only comment", False)
invalid_scenario = scenario.process(invalid_step_order_example_text, False)
pass

In this case the `Scenario` dataclass instances may be malformed. (See [Data structure](data.ipynb) for details about the dataclass structure.)

In [7]:
print(str(empty_scenario)) # Scenario:


Scenario: 



In [8]:
print(str(comment_only_scenario)) # Scenario:


Scenario: 



In [9]:
print(str(invalid_scenario)) # Scenario: Make pizza
                             # Given we knead the dough into a <Size> wide pizza disc
                             # Given we put the ingredients on the pizza disc in the following order:
                             # | Order      |
                             # | <Sauce>    |
                             # | <Toppings> |
                             # Then the <Type> pizza is ready to eat
                             # When we cook the pizza


Scenario: Make pizza
Given we knead the dough into a <Size> wide pizza disc
Given we put the ingredients on the pizza disc in the following order:
| Order      |
| <Sauce>    |
| <Toppings> |
Then the <Type> pizza is ready to eat
When we cook the pizza


## Input & Output

The utilities support basic input and output operations.

### Input

A file location can be passed as a string and the `scenario.load(str, bool=True): Scenario` function will attempt to load and process the file. The second parameter determines the validation of the scenario.

In [10]:
loaded_example = scenario.load("examples/data/simple_example.feature")
print(f"{type(loaded_example)}\n{loaded_example}") # <class 'gherkin_processor.scenario.Scenario'>
                                                   # Scenario: Make hotdog
                                                   # Given the sausage is cooked
                                                   # When the sausage is put into a bun
                                                   # Then the hotdog is ready to eat


<class 'gherkin_processor.scenario.Scenario'>

Scenario: Make hotdog
Given the sausage is cooked
When the sausage is put into a bun
Then the hotdog is ready to eat


### Output

The utilities can save a scenario in a text format or in a JSON format.

#### Text output

This will create a valid syntax Gherkin scenario into a given file.

In [11]:
scenario.save(scenario.process(simple_example_text), "examples/data/saved_example.feature")

In [12]:
# print the created file
! cat examples/data/saved_example.feature
# remove the created file
! rm examples/data/saved_example.feature


Scenario: Make hotdog
Given the sausage is cooked
When the sausage is put into a bun
Then the hotdog is ready to eat


#### JSON output

This will create a json format extraction of the Gherkin scenario into a given file.

In [13]:
scenario.save_as_json(scenario.process(simple_example_text), "examples/data/saved_example.json")

In [14]:
# print the created file
! cat examples/data/saved_example.json
# remove the created file
! rm examples/data/saved_example.json

{
    "tags": [],
    "name": "Make hotdog",
    "steps": [
        {
            "step": "Given",
            "description": "the sausage is cooked"
        },
        {
            "step": "When",
            "description": "the sausage is put into a bun"
        },
        {
            "step": "Then",
            "description": "the hotdog is ready to eat"
        }
    ],
    "template_table": null
}
